In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import argparse
from workalendar.asia import China # 节假日计算包

In [2]:
filepath = 'D:/Code/SF/SF_Technology_Algorithm/data/quantity_train.csv'
ori_data = pd.read_csv(filepath)
ori_data.head()

,date,aoi_id,pai_num,shou_num,zone_id,aoi_area,aoi_type
0,20200301,aoi_106,6,5,zone_0,2109.0,住宅小区
1,20200301,aoi_131,8,4,zone_0,2366.0,CBD及商住两用楼宇
2,20200301,aoi_14,10,2,zone_0,7126.0,住宅小区
3,20200301,aoi_147,4,2,zone_0,1947.0,住宅小区
4,20200301,aoi_174,47,204,zone_0,8359.0,购物中心


对收派件量做处理，填充无收派件的日期的量为0

In [33]:
target_collects = pd.pivot_table(data=ori_data,
                                    index = ['zone_id'],
                                     columns = ['date'],
                                     values = ['pai_num'],
                                     aggfunc= 'sum')
#target_collects.fillna(0, inplace=True)
target_collects = target_collects.stack()
target_collects.reset_index(inplace=True)
target_collects

,zone_id,date,pai_num
0,zone_0,20200301,481.0
1,zone_0,20200302,478.0
2,zone_0,20200303,603.0
3,zone_0,20200304,565.0
4,zone_0,20200305,545.0
...,...,...,...
4873,zone_9,20200806,449.0
4874,zone_9,20200807,474.0
4875,zone_9,20200808,364.0
4876,zone_9,20200809,334.0


,index,zone_id,date,pai_num,shou_num
0,0,zone_0,20200301,481.0,1062.0
1,1,zone_0,20200302,478.0,470.0
2,2,zone_0,20200303,603.0,476.0
3,3,zone_0,20200304,565.0,437.0
4,4,zone_0,20200305,545.0,440.0
...,...,...,...,...,...
4873,4885,zone_9,20200806,449.0,242.0
4874,4886,zone_9,20200807,474.0,259.0
4875,4887,zone_9,20200808,364.0,91.0
4876,4888,zone_9,20200809,334.0,100.0


In [52]:
a = pd.Series([1,2,3,4])
a[3:]

3    4
dtype: int64

In [5]:
data_pivot_shou_num = pd.pivot_table(data=ori_data,
                                    index = ['zone_id'],
                                     columns = ['date_str'],
                                     values = ['shou_num'],
                                     aggfunc={'shou_num':'sum'})
data_pivot_shou_num.fillna(0,inplace = True)
data_pivot_shou_num = data_pivot_shou_num.unstack()
data_pivot_shou_num
data_pivot_shou_num.name = 'shou_num'
data_pivot_shou_num = data_pivot_shou_num.reset_index()

#### 对面积做处理，各zone_id取其总面积最大的那个日期为其总面积

In [6]:
data_pivot_area = pd.pivot_table(data=ori_data,
                                    index = ['zone_id'],
                                     columns = ['date_str'],
                                     values = ['aoi_area'],
                                     aggfunc={'aoi_area':'sum'})
data_pivot_area.fillna(0,inplace = True)
data_pivot_area = data_pivot_area.max(axis=1)
data_pivot_area.name = 'area'
data_pivot_area = data_pivot_area.reset_index()

#### 对各zone所含的aoi数量、种类数做处理，取其所含最多的那天的值

In [7]:
data_pivot_aoi_num = pd.pivot_table(data=ori_data,
                                     index = ['zone_id'],
                                     columns = ['date_str'],
                                     values = ['aoi_id'],
                                     aggfunc={'aoi_id':'count'})
data_pivot_aoi_num.fillna(0,inplace = True)
data_pivot_aoi_num = data_pivot_aoi_num.max(axis=1)
data_pivot_aoi_num.name = 'aoi_num'
data_pivot_aoi_num = data_pivot_aoi_num.reset_index()

In [8]:
data_pivot_aoi_type = pd.pivot_table(data=ori_data,
                                     index = ['zone_id'],
                                     columns = ['date_str'],
                                     values = ['aoi_type'],
                                     aggfunc={'aoi_type':lambda x :len(x.unique())})
data_pivot_aoi_type.fillna(0,inplace = True)
data_pivot_aoi_type = data_pivot_aoi_type.max(axis=1)
data_pivot_aoi_type.name = 'aoi_type'
data_pivot_aoi_type = data_pivot_aoi_type.reset_index()

#### 将收派件数量、最大总和面积、最多aoi数目、最多aoi_type数目合并为一个DataFrame

In [9]:
df_shou_pai = pd.merge(left = data_pivot_pai_num , right=data_pivot_shou_num, on=['zone_id','date_str'])
df_shou_pai.drop(['level_0_x','level_0_y'],axis=1,inplace=True)

In [10]:
df_shou_pai_area = pd.merge(df_shou_pai, data_pivot_area, on = ['zone_id'])

In [11]:
df_shou_pai_area_num = pd.merge(df_shou_pai_area, data_pivot_aoi_num, on = ['zone_id'])

In [12]:
df_shou_pai_area_num_type = pd.merge(df_shou_pai_area_num, data_pivot_aoi_type, on=['zone_id'])
df = df_shou_pai_area_num_type
df['date_str'] = df['date_str'].astype(str)

#### 获取法定节假日信息

In [22]:
def cal_festival(year):
    cal = China()
    lis = []
    if type(year) != list:   # eval函数就是实现list、dict、tuple与str之间的转化
        year = eval(year)
    for ye in year:
        for x, v in cal.holidays(ye):
            lis.append([str(x).replace('-', ''), v])
    lis.append(['20190504', 'Labour Day Holiday'])
    df = pd.DataFrame(data=lis, columns=['date', 'festival'])
    return df

df2 = cal_festival([2020])
df2

D:\Anaconda\lib\site-packages\workalendar\asia\china.py:81: UserWarning: Support years 2018-2020 currently, need update every year.
  year_max,


,date,festival
0,20200101,New year
1,20200124,Chinese New Year's eve
2,20200125,Chinese New Year
3,20200126,Spring Festival
4,20200127,Spring Festival
5,20200128,Spring Festival
6,20200129,Spring Festival
7,20200130,Spring Festival
8,20200404,Ching Ming Festival
9,20200405,Ching Ming Festival


In [14]:
df = pd.merge(df, df2, left_on = 'date_str', right_on = 'date', how = 'left')
df.drop('date',axis=1,inplace=True)
df

,date_str,zone_id,pai_num,shou_num,area,aoi_num,aoi_type,festival
0,20200301,zone_0,481.0,1062.0,286883.06033,21.0,6.0,NaN
1,20200302,zone_0,478.0,470.0,286883.06033,21.0,6.0,NaN
2,20200303,zone_0,603.0,476.0,286883.06033,21.0,6.0,NaN
3,20200304,zone_0,565.0,437.0,286883.06033,21.0,6.0,NaN
4,20200305,zone_0,545.0,440.0,286883.06033,21.0,6.0,NaN
...,...,...,...,...,...,...,...,...
4885,20200806,zone_9,449.0,242.0,402486.23948,14.0,4.0,NaN
4886,20200807,zone_9,474.0,259.0,402486.23948,14.0,4.0,NaN
4887,20200808,zone_9,364.0,91.0,402486.23948,14.0,4.0,NaN
4888,20200809,zone_9,334.0,100.0,402486.23948,14.0,4.0,NaN


#### 转换日期的格式

In [16]:
dt_format = '%Y%m%d'
df['date_str'] = pd.to_datetime(df['date_str'], format=dt_format)
df.columns = ['date','zone_id','pai_num','shou_num','area','aoi_num','aoi_type', 'festtival']

In [21]:
df.sort_values(by=['zone_id','date'],ascending=(True,True))

,date,zone_id,pai_num,shou_num,area,aoi_num,aoi_type,festtival
0,2020-03-01,zone_0,481.0,1062.0,286883.06033,21.0,6.0,NaN
1,2020-03-02,zone_0,478.0,470.0,286883.06033,21.0,6.0,NaN
2,2020-03-03,zone_0,603.0,476.0,286883.06033,21.0,6.0,NaN
3,2020-03-04,zone_0,565.0,437.0,286883.06033,21.0,6.0,NaN
4,2020-03-05,zone_0,545.0,440.0,286883.06033,21.0,6.0,NaN
...,...,...,...,...,...,...,...,...
4885,2020-08-06,zone_9,449.0,242.0,402486.23948,14.0,4.0,NaN
4886,2020-08-07,zone_9,474.0,259.0,402486.23948,14.0,4.0,NaN
4887,2020-08-08,zone_9,364.0,91.0,402486.23948,14.0,4.0,NaN
4888,2020-08-09,zone_9,334.0,100.0,402486.23948,14.0,4.0,NaN


In [17]:
data_pivot['zone_id'] = data_pivot['zone_id'].apply(lambda x: x.split('_')[1])
data_pivot.head()

,date_str,zone_id,aoi_area,aoi_type,pai_num,shou_num
0,2020-03-01,0,14253.503352,5,481,1062
1,2020-03-01,1,31889.205640,5,235,290
2,2020-03-01,10,3545.992463,3,229,482
3,2020-03-01,11,6991.188420,3,248,169
4,2020-03-01,12,17549.579236,6,523,209


In [20]:
time_series = data_pivot['date_str']
date_normal_features = np.column_stack((time_series.dt.year,
                                                time_series.dt.month,
                                                time_series.dt.day,
                                                time_series.dt.weekofyear,
                                                time_series.dt.weekday,
                                                time_series.dt.dayofyear,
                                                time_series.dt.quarter,
                                                time_series.dt.weekday.isin([5, 6]),
                                                time_series.dt.is_month_start,
                                                time_series.dt.is_month_end,
                                                time_series.dt.is_quarter_start,
                                                time_series.dt.is_quarter_end))

In [21]:
date_normal_features


array([[2020,    3,    1, ...,    0,    0,    0],
       [2020,    3,    1, ...,    0,    0,    0],
       [2020,    3,    1, ...,    0,    0,    0],
       ...,
       [2020,    8,   10, ...,    0,    0,    0],
       [2020,    8,   10, ...,    0,    0,    0],
       [2020,    8,   10, ...,    0,    0,    0]])

In [30]:
#获取傅里叶数据
def fourier_series(date_series, period, series_order):
    t = np.array(
        (date_series - pd.datetime(1970, 1, 1))
            .dt.total_seconds()
            .astype(np.float)
    ) / (3600 * 24.)

    value = np.column_stack(
        [fun((2.0 * (i + 1) * np.pi * t / period)) for i in range(series_order) for fun in (np.sin, np.cos)])
    return value
daily_features=fourier_series(data_pivot['date_str'], period=1, series_order=3)

<ipython-input-30-7b71a8664e4a>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  (date_series - pd.datetime(1970, 1, 1))


,daily_delim_1,daily_delim_2,daily_delim_3,daily_delim_4,daily_delim_5,daily_delim_6
0,-1.169250e-11,1.0,-2.338500e-11,1.0,-2.052558e-11,1.0
1,-1.169250e-11,1.0,-2.338500e-11,1.0,-2.052558e-11,1.0
2,-1.169250e-11,1.0,-2.338500e-11,1.0,-2.052558e-11,1.0
3,-1.169250e-11,1.0,-2.338500e-11,1.0,-2.052558e-11,1.0
4,-1.169250e-11,1.0,-2.338500e-11,1.0,-2.052558e-11,1.0
...,...,...,...,...,...,...
4873,7.307420e-13,1.0,1.461484e-12,1.0,2.192226e-12,1.0
4874,7.307420e-13,1.0,1.461484e-12,1.0,2.192226e-12,1.0
4875,7.307420e-13,1.0,1.461484e-12,1.0,2.192226e-12,1.0
4876,7.307420e-13,1.0,1.461484e-12,1.0,2.192226e-12,1.0


In [1]:
import pandas as pd
import argparse
from workalendar.asia import China # 节假日计算包

def cal_festival(year):
    cal = China()
    lis = []
    if type(year) != list:   # eval函数就是实现list、dict、tuple与str之间的转化
        year = eval(year)
    for ye in year:
        for x, v in cal.holidays(ye):
            lis.append([str(x).replace('-', ''), v])
    lis.append(['20190504', 'Labour Day Holiday'])
    df = pd.DataFrame(data=lis, columns=['date', 'festival'])
    return df

df2 = cal_festival([2020])
df2

/Users/perry/opt/anaconda3/lib/python3.8/site-packages/workalendar/asia/china.py:78: UserWarning: Support years 2018-2020 currently, need update every year.
  warnings.warn(


,date,festival
0,20200101,New year
1,20200124,Chinese New Year's eve
2,20200125,Chinese New Year
3,20200126,Spring Festival
4,20200127,Spring Festival
5,20200128,Spring Festival
6,20200129,Spring Festival
7,20200130,Spring Festival
8,20200404,Ching Ming Festival
9,20200405,Ching Ming Festival


In [31]:
target_columns=['pai_num', 'shou_num']
a = target_columns.remove('pai_num')

In [35]:
a = pd.DataFrame(np.arange(9).reshape(3,3))
a

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [53]:
a.columns=['a','b','c']
a
print(a.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       3 non-null      int32
 1   b       3 non-null      int32
 2   c       3 non-null      int32
dtypes: int32(3)
memory usage: 164.0 bytes
None


In [52]:
pd.Series(a['a'].values)

0    0
1    3
2    6
dtype: int32

In [54]:
b = a.iloc[:,-1]
b

0    2
1    5
2    8
Name: c, dtype: int32

In [42]:
for id, groups in ori_data.groupby('zone_id'):
    print(id)
    print(groups.iloc[-3:,])

zone_0
      date_str  aoi_id  pai_num  shou_num zone_id   aoi_area    aoi_type
2839  20200810  aoi_50        5         7  zone_0   2088.000  CBD及商住两用楼宇
2840  20200810  aoi_86       74        92  zone_0  16258.000        住宅小区
2841  20200810  aoi_94       12         6  zone_0   9688.992        宾馆酒店
zone_1
      date_str   aoi_id  pai_num  shou_num zone_id    aoi_area    aoi_type
3652  20200810  aoi_321        2         5  zone_1  77110.6800        风景名胜
3653  20200810   aoi_69      149       166  zone_1   3889.9622  CBD及商住两用楼宇
3654  20200810   aoi_84       23        40  zone_1  35720.0000        购物中心
zone_10
      date_str   aoi_id  pai_num  shou_num  zone_id    aoi_area    aoi_type
5046  20200810  aoi_298       37       228  zone_10   2813.5146  CBD及商住两用楼宇
5047  20200810  aoi_302       22        55  zone_10  11061.0000        宾馆酒店
5048  20200810  aoi_306       59        61  zone_10   4682.2373  CBD及商住两用楼宇
zone_11
      date_str   aoi_id  pai_num  shou_num  zone_id  aoi_area    aoi_type


In [46]:
x = np.array([[1],[2],[3]])
x

array([[1],
       [2],
       [3]])

In [48]:
x.reshape(1,3)

array([[1, 2, 3]])